In [1]:
from datasets import load_dataset

ModuleNotFoundError: No module named 'datasets'

In [ ]:
dataset = load_dataset("squad")
dataset

: 

In [2]:
def add_end_of_text(example):
    example["question"] = example["question"] + "<|endoftext|>"
    return example

dataset = dataset.remove_columns(['id', 'title', 'context', 'answers'])
dataset.map(add_end_of_text)


NameError: name 'dataset' is not defined

In [ ]:
dataset['train']['question'][:10]

: 

In [18]:
from transformers import AutoTokenizer
model_checkpoint = "t5"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

In [16]:
sequence = ("This is a tokenizer that is being applied to a sequence" 
            + "at Harvard University.|<endoftext>|")
tokens = tokenizer.tokenize(sequence)
print(tokens)

['This', 'Ġis', 'Ġa', 'Ġtoken', 'izer', 'Ġthat', 'Ġis', 'Ġbeing', 'Ġapplied', 'Ġto', 'Ġa', 'Ġsequence', 'at', 'ĠHarvard', 'ĠUniversity', '.', '|', '<', 'end', 'of', 'text', '>', '|']


In [6]:
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids) # one per token

[1212, 318, 257, 11241, 7509, 326, 318, 852, 5625, 284, 257, 8379, 265, 11131, 2059, 13, 91, 27, 437, 1659, 5239, 29, 91]


In [7]:
tokenizer(sequence)

{'input_ids': [1212, 318, 257, 11241, 7509, 326, 318, 852, 5625, 284, 257, 8379, 265, 11131, 2059, 13, 91, 27, 437, 1659, 5239, 29, 91], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
def tokenize_function(examples):
    return tokenizer[examples["question"], truncation=True]

tokenized_dataset = dataset.map(
    tokenize_function,
    
)

In [ ]:
tokenized_datasets

In [ ]:
s1 = "I went to the yard"
s2 = "I went to the park and got myself a burger"

m1 = "I went to the yard </2> I went"
m2 = "to the park and got myself"

In [ ]:
block_size = 128

def group_texts(examples):
    # repeat concatenation for each of the keys in the dictionary.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    
    # populate each of the input_ids and other keys
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    
    # add labels
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4
)

In [ ]:
small_train_dataset = \
    lm_datasets["train"].shuffle(seed=42).select(range(100))

small_eval_dataset = \
    lm_datasets["validation"].shuffle(seed=42).select(range(100))

In [ ]:
# creating the model

In [ ]:
training_args = TrainingArguments(
    output_dir="./distilgpt2-squad",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset
)

trainer.train()